In [1]:
from gensim.models import Word2Vec
import matplotlib.pyplot as plt
import numpy as np

In [1]:
from dataset import build_corpus
data = build_corpus(1000, return_fields=['corpus', 'word2idx', 'idx2word', 'word_count'],  load=True)

/home/lorenzo/anaconda3/envs/gensim/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [79]:
model = Word2Vec.load('models/model_100k_300_4_25_1.model')

In [ ]:
model.wv.most_similar('computer', topn=5)

In [ ]:
model.wv.most_similar(positive=['king', 'woman'], negative=['man'], topn=5)

In [7]:
def norm(embedding):
    return np.linalg.norm(embedding)

In [8]:
arq = open('kl-100000.txt', 'r')
content = arq.read()
arq.close()

In [9]:
kl_divergences = eval(content)

In [16]:
from dataset import build_corpus
data = build_corpus(1000, return_fields=['corpus', 'word2idx', 'idx2word', 'word_count'],  load=True)

In [17]:
corpus, w2idx, idx2w, wc = data['corpus'], data['word2idx'], data['idx2word'], data['word_count']

In [18]:
filtered_words = {key: value for key, value in wc.items() if value >=10}

## Visualizing the relation

In [19]:
data = [(norm(model.wv[word]) if word in model.wv else 0, kl_divergences[word]) for word in filtered_words.keys()]

NameError: name 'model' is not defined

In [60]:
y = [data[i][1] for i in range(len(data)) if data[i][0] != 0]
x = [data[i][0] for i in range(len(data)) if data[i][0] != 0]  # Todos os valores de norm(model.wv[word])

In [ ]:
plt.figure(figsize=(10, 6))

# Scatter plot com personalização
scatter = plt.scatter(x, y, c=y, cmap='viridis', alpha=0.8, edgecolors='k', s=50)

# Adicionando título e rótulos
plt.title("Relação entre Normas de Vetores e Divergência KL", fontsize=14)
plt.xlabel("Norma do Vetor", fontsize=12)
plt.ylabel("Divergência KL", fontsize=12)

# Adicionando uma barra de cores para interpretação
cbar = plt.colorbar(scatter)
cbar.set_label("Divergência KL", fontsize=12)

# Adicionando uma grade para facilitar a leitura
plt.grid(True, linestyle='--', alpha=0.6)

# Exibindo o gráfico
plt.show()

## Regression

In [ ]:
from sklearn.linear_model import LinearRegression

In [ ]:
X = [item[0] for item in data]  # Todos os valores de norm(model.wv[word])
y = [item[1] for item in data]

In [ ]:
model = LinearRegression()
model.fit(X, y)

In [ ]:
print("Coeficiente:", model.gpt)